In [1]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import utils
import cv2

In [2]:
# import the necessary packages
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import imutils
import cv2
 
def midpoint(ptA, ptB):
	return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

In [7]:
# load the image, convert it to grayscale, and blur it slightly
image = cv2.imread("origin2.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), cv2.BORDER_DEFAULT)
 
# perform edge detection, then perform a dilation + erosion to
# close gaps in between object edges
edged = cv2.Canny(gray, 50, 100)
edged = cv2.dilate(edged, None, iterations=1)
edged = cv2.erode(edged, None, iterations=1)
 
# find contours in the edge map
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
 
# sort the contours from left-to-right and, then initialize the
# distance colors and reference object
#(cnts, _) = contours.sort_contours(cnts, method="top-to-bottom")
colors = ((0, 0, 255), (240, 0, 159), (0, 165, 255), (255, 255, 0),(255, 0, 255))
refObj = None

In [9]:
# loop over the contours individually, and 1-1, 1-1, 1-1,...
orig = image.copy()
for c in cnts:
    # if the contour is too large, ignore it
    if cv2.contourArea(c) > 40:
        refObj = None
        #refObj = (box, (cX, cY), D /1)
        continue
    # compute the rotated bounding box of the contour
    box = cv2.minAreaRect(c)
    box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
    box = np.array(box, dtype="int")

    # order the points in the contour such that they appear
    # in top-left, top-right, bottom-right, and bottom-left
    # order, then draw the outline of the rotated bounding
    # box
    box = perspective.order_points(box)

    # compute the center of the bounding box
    cX = np.average(box[:, 0])
    cY = np.average(box[:, 1])
    
    # skip if not white
    if image[int(cY),int(cX)][0] < 150 or image[int(cY),int(cX)][1] < 150 or image[int(cY),int(cX)][2] < 150:
        refObj = None
        #refObj = (box, (cX, cY), D /1)
        continue
        
    print(cX,',',cY)
    print(image[int(cY),int(cX)])
    
    if refObj is None:
        # unpack the ordered bounding box, then compute the
        # midpoint between the top-left and top-right points,
        # followed by the midpoint between the top-right and
        # bottom-right
        (tl, tr, br, bl) = box
        (tlblX, tlblY) = midpoint(tl, bl)
        (trbrX, trbrY) = midpoint(tr, br)

        # compute the Euclidean distance between the midpoints,
        # then construct the reference object
        D = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
        refObj = (box, (cX, cY), D /1) # 1 is the diameter of the reference object, requires manual inputs 
        continue
    # draw the contours on the image
    cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 1)
    cv2.drawContours(orig, [refObj[0].astype("int")], -1, (0, 255, 0), 1)

    # stack the reference coordinates and the object coordinates
    # to include the object center
    refCoords = np.vstack([refObj[0], refObj[1]])
    objCoords = np.vstack([box, (cX, cY)])
    
    # loop over the original points
    for ((xA, yA), (xB, yB), color) in zip(refCoords, objCoords, colors):
        # draw circles corresponding to the current points and
        # connect them with a line
        if color == (255, 0, 255):
            # compute the Euclidean distance between the coordinates,
            D = dist.euclidean((xA, yA), (xB, yB)) / refObj[2]
            if D > 15:
                refObj = None
                #refObj = (box, (cX, cY), D /1)
                continue
            print(D)
            cv2.circle(orig, (int(xA), int(yA)), 1, color, -1)
            cv2.circle(orig, (int(xB), int(yB)), 1, color, -1)
            cv2.line(orig, (int(xA), int(yA)), (int(xB), int(yB)),color, 1)
           
            # and then convert the distance in pixels to distance in units
            (mX, mY) = midpoint((xA, yA), (xB, yB))
            cv2.putText(orig, "{:.1f}cm".format(D), (int(mX), int(mY - 10)),cv2.FONT_HERSHEY_SIMPLEX, 0.25, color, 1)
    refObj = None
    #refObj = (box, (cX, cY), D /1)
# show the output image
cv2.imwrite("output2.png", orig)

188.0 , 656.0
[183 187 192]
268.5 , 640.5
[198 200 201]
13.663108825683594
176.0 , 635.0
[202 210 219]
256.5 , 615.5
[186 179 196]
190.0 , 613.0
[201 211 205]
13.309394836425781
263.0 , 591.0
[234 230 223]
181.0 , 589.0
[213 218 212]
13.670730590820312
252.5 , 566.0
[178 185 176]
190.0 , 566.0
[188 183 195]
12.5
272.0 , 565.0
[172 184 183]
255.5 , 541.5
[228 240 238]
4.785684585571289
185.5 , 540.5
[237 236 237]
207.0 , 539.5
[151 164 174]
3.074748992919922
269.0 , 513.5
[212 212 218]
246.5 , 514.0
[218 222 217]
3.7509256998697915
198.5 , 513.0
[219 230 241]
177.5 , 511.5
[215 213 204]
3.007643290928432
257.0 , 488.5
[227 224 233]
279.0 , 486.0
[210 207 211]
211.0 , 486.0
[228 227 230]
11.333333333333334
189.5 , 484.0
[241 243 244]
261.0 , 462.5
[236 235 237]
10.666082109723773
242.5 , 463.0
[215 216 215]
214.0 , 460.5
[219 217 223]
5.7218879699707035
283.0 , 458.5
[211 210 213]
194.0 , 457.5
[252 252 248]
14.834269205729166
175.5 , 453.5
[215 215 206]
232.0 , 440.0
[215 214 218]
11.61

True